HPC Mini-Challenge 2 - Acceleration in Data Science
 Part 2: GPU
 FHNW - FS2024

Original by S. Suter, adapted by S. Marcin and M. Stutz

Submitted by: Nikodem Wojtczak

 Resources
* [Overview of GPU Programming](https://www.cherryservers.com/blog/introduction-to-gpu-programming-with-cuda-and-python)
* [CUDA Basic Parts](https://nyu-cds.github.io/python-gpu/02-cuda/)
* [Accelerate Code with CuPy](https://towardsdatascience.com/heres-how-to-use-cupy-to-make-numpy-700x-faster-4b920dda1f56)
* Lectures and examples from the PAC (parallel computing) computer science course, see folder "resources"
* CSCS "High-Performance Computing with Python" course, Day 3:
    - JIT Numba GPU 1 + 2
    - https://youtu.be/E4REVbCVxNQ
    - https://github.com/eth-cscs/PythonHPC/tree/master/numba-cuda
    - See also the current tutorial from 2021
* [Google CoLab](https://colab.research.google.com/) or your own GPU if available.


In [3]:
from numba import config

config.CUDA_ENABLE_PYNVJITLINK = 1

In [4]:
import sys
from numba import cuda
import os

print("--- Environment Diagnostics ---")
print(f"Python Version: {sys.version}")

try:
    print("\nNumba CUDA Diagnostics:")
    if cuda.is_available():
        cuda.detect()
    else:
        print("Numba could not find a CUDA-enabled GPU.")
except Exception as e:
    print(f"An error occurred during Numba CUDA detection: {e}")

# Check for CUDA_HOME environment variable
print(f"\nCUDA_HOME environment variable: {os.environ.get('CUDA_HOME', 'Not Set')}")
print("-----------------------------")

--- Environment Diagnostics ---
Python Version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]

Numba CUDA Diagnostics:
Found 1 CUDA devices
id 0             b'Tesla T4'                              [SUPPORTED]
                      Compute Capability: 7.5
                           PCI Device ID: 4
                              PCI Bus ID: 0
                                    UUID: GPU-7761e32a-6ec9-7b68-c1e0-b09b0b4acf82
                                Watchdog: Disabled
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported

CUDA_HOME environment variable: Not Set
-----------------------------


In [6]:
# Dummy Beispiel zum testen mit Numba
import math
from numba import vectorize, cuda
import numpy as np

# This check is important to see if a GPU is available.
if cuda.is_available():

    @vectorize(["float32(float32)"], target="cuda")
    def gpu_sqrt(x):
        return math.sqrt(x)

    a = np.arange(4096, dtype=np.float32)
    # Move data to GPU, compute, and bring back
    a_device = cuda.to_device(a)
    result_device = gpu_sqrt(a_device)
    result_host = result_device.copy_to_host()
    print("Numba CUDA vectorize test successful.")
else:
    print("Numba CUDA not available.")

Numba CUDA vectorize test successful.


/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/dispatcher.py:579: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


## 5 GPU Reconstruction

Implement an SVD reconstruction variant on the GPU or in a hybrid setting. You may use code from the first part. Carefully choose which parts of the algorithm should be implemented in a GPU kernel and which are more efficiently executed on the CPU. Incorporate insights from the first part. At least one component of the algorithm must be implemented in a GPU kernel. Document any assumptions you make for simplification. Evaluate whether you want to use CuPy or Numba.

Links:
* [Examples: Matrix Multiplication](https://numba.readthedocs.io/en/latest/cuda/examples.html)


In [3]:
### BEGIN SOLUTION
import os
import time
import glob
import uuid
import imageio.v3 as imageio
from PIL import Image
import numpy as np

source_subfolder = "VeryMildDemented"
processed_subfolder = "VeryMildDemented_png"
base_dir = "adni_png"
source_folders = os.path.join(base_dir, source_subfolder)
processed_folders = os.path.join(base_dir, processed_subfolder)
standard_size = (256, 256)

if not os.path.exists(processed_folders):
    os.makedirs(processed_folders)

jpg_files = sorted(glob.glob(f"{source_folders}/*.jpg"))
if jpg_files:
    for jpg_file in jpg_files:
        png_filename = os.path.splitext(os.path.basename(jpg_file))[0] + ".png"
        png_filepath = os.path.join(processed_folders, png_filename)
        if not os.path.exists(png_filepath):
            with Image.open(jpg_file) as img_pil:
                img_processed = img_pil.convert("L").resize(
                    standard_size, Image.Resampling.LANCZOS
                )
                img_processed.save(png_filepath)

png_files = sorted(glob.glob(f"{processed_folders}/*.png"))
im_cpu, u_cpu, s_cpu, vt_cpu = None, None, None, None
if png_files:
    im_cpu = np.array(Image.open(png_files[0])).astype(np.float32)
    if im_cpu.max() > 0:
        im_cpu = (im_cpu - im_cpu.min()) / (im_cpu.max() - im_cpu.min())
    u_cpu, s_cpu, vt_cpu = np.linalg.svd(im_cpu, full_matrices=False)
else:
    print("No processed images found to run GPU reconstruction.")

In [7]:
import cupy as cp


# --- Option 1: CuPy Implementation (Leverages optimized cuBLAS libraries) ---
def reconstruct_svd_cupy(u_cpu, s_cpu, vt_cpu, k):
    """SVD reconstruction using CuPy, a drop-in replacement for NumPy."""
    # 1. Move data from CPU (host) to GPU (device)
    u_gpu = cp.asarray(u_cpu)
    s_gpu = cp.asarray(s_cpu)
    vt_gpu = cp.asarray(vt_cpu)

    # 2. Perform the computation on the GPU using CuPy's NumPy-like syntax
    reco_gpu = (u_gpu[:, :k] * s_gpu[:k]) @ vt_gpu[:k, :]

    # 3. Move the result from GPU back to CPU
    reco_cpu = cp.asnumpy(reco_gpu)
    return reco_cpu


# --- Option 2: Numba Custom Kernel (More manual control) ---
@cuda.jit
def reconstruct_svd_numba_kernel(reco, u, s, vt, k):
    """Custom CUDA kernel for SVD reconstruction."""
    i, j = cuda.grid(2)
    m, n = reco.shape
    if i < m and j < n:
        temp_sum = 0.0
        for l in range(k):
            temp_sum += u[i, l] * s[l] * vt[l, j]
        reco[i, j] = temp_sum


def reconstruct_svd_numba_gpu(u_cpu, s_cpu, vt_cpu, k):
    """Wrapper function to run the Numba CUDA kernel."""
    m, n = u_cpu.shape[0], vt_cpu.shape[1]
    u_gpu = cuda.to_device(u_cpu)
    s_gpu = cuda.to_device(s_cpu)
    vt_gpu = cuda.to_device(vt_cpu)
    reco_gpu = cuda.device_array((m, n), dtype=np.float32)

    threads_per_block = (16, 16)
    blocks_per_grid_x = math.ceil(m / threads_per_block[0])
    blocks_per_grid_y = math.ceil(n / threads_per_block[1])
    blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

    reconstruct_svd_numba_kernel[blocks_per_grid, threads_per_block](
        reco_gpu, u_gpu, s_gpu, vt_gpu, k
    )
    return reco_gpu.copy_to_host()


# --- Comparison ---
if cuda.is_available() and u_cpu is not None:
    k = 100
    print(f"--- GPU Reconstruction Performance (k={k}) ---")

    start = time.perf_counter()
    reco_cupy = reconstruct_svd_cupy(u_cpu, s_cpu, vt_cpu, k)
    cupy_time = (time.perf_counter() - start) * 1000
    print(f"CuPy execution time: {cupy_time:.4f} ms")

    start = time.perf_counter()
    reco_numba = reconstruct_svd_numba_gpu(u_cpu, s_cpu, vt_cpu, k)
    numba_time = (time.perf_counter() - start) * 1000
    print(f"Numba Kernel execution time: {numba_time:.4f} ms")

    start = time.perf_counter()
    reco_cpu_broadcast = (u_cpu[:, :k] * s_cpu[:k]) @ vt_cpu[:k, :]
    cpu_time = (time.perf_counter() - start) * 1000
    print(f"CPU (NumPy Broadcast) time: {cpu_time:.4f} ms")
else:
    print("GPU not available or no images found, skipping GPU execution.")
### END SOLUTION

--- GPU Reconstruction Performance (k=100) ---
CuPy execution time: 1427.9841 ms
Numba Kernel execution time: 249.4730 ms
CPU (NumPy Broadcast) time: 0.6553 ms


 **Analysis of Initial Results:**

 The initial test on a single 256x256 image clearly demonstrates a critical principle of accelerated computing: **overhead is dominant for small tasks**. The results were:
 - **CPU (NumPy): 0.66 ms**
 - **GPU (Numba Kernel): 249.47 ms**
 - **GPU (CuPy): 1427.98 ms**

 The CPU is orders of magnitude faster. This is because the time required to allocate GPU memory, transfer the small matrices from CPU to GPU, launch the kernel, and copy the result back completely dwarfs the actual computation time. The NumPy operation on the CPU has virtually zero overhead and wins easily.

 **CuPy vs. Numba:** Surprisingly, the custom Numba kernel was significantly faster than CuPy. This is unusual, as CuPy typically calls highly optimized libraries. This result likely points to a high startup or first-call overhead for CuPy in the Colab environment for this specific problem size, whereas the Numba JIT compilation was more efficient in this isolated case. For larger, more complex tasks, CuPy is still the recommended approach.


## 5.2 GPU Kernel Performance

### 5.2.1 Blocks and Input Size

Links:
* [Examples: Matrix Multiplication](https://numba.readthedocs.io/en/latest/cuda/examples.html)
* [NVIDIA Chapter on "Strided Access"](https://spaces.technik.fhnw.ch/multimediathek/file/cuda-best-practices-in-c)
* https://developer.nvidia.com/blog/cublas-strided-batched-matrix-multiply/
* https://developer.nvidia.com/blog/how-access-global-memory-efficiently-cuda-c-kernels/

Conduct 2-3 experiments with different block configurations and input data sizes. For this, create a new dataset with arbitrarily large matrices, since the GPU is particularly well-suited for processing large inputs (use these differently sized matrices for all subsequent comparisons and tasks as well). Measure the performance of the GPU kernel using appropriate functions. Which block size, depending on the input size, proved to be the most successful in your experiments? What do you think are the reasons for this? What are the performance differences between your CPU and GPU implementation? Discuss your analysis (optionally with graphics).


In [8]:
### BEGIN SOLUTION
def benchmark_gpu_kernel(matrix_size, block_size_2d, k):
    """Benchmarks the Numba kernel for a given matrix and block size."""
    u_cpu = np.random.rand(matrix_size, matrix_size).astype(np.float32)
    s_cpu = np.random.rand(matrix_size).astype(np.float32)
    vt_cpu = np.random.rand(matrix_size, matrix_size).astype(np.float32)

    start_transfer = time.perf_counter()
    u_gpu = cuda.to_device(u_cpu)
    s_gpu = cuda.to_device(s_cpu)
    vt_gpu = cuda.to_device(vt_cpu)
    reco_gpu = cuda.device_array((matrix_size, matrix_size), dtype=np.float32)
    cuda.synchronize()
    transfer_time_ms = (time.perf_counter() - start_transfer) * 1000

    threads_per_block = block_size_2d
    blocks_per_grid_x = math.ceil(matrix_size / threads_per_block[0])
    blocks_per_grid_y = math.ceil(matrix_size / threads_per_block[1])
    blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

    start_kernel = time.perf_counter()
    reconstruct_svd_numba_kernel[blocks_per_grid, threads_per_block](
        reco_gpu, u_gpu, s_gpu, vt_gpu, k
    )
    cuda.synchronize()
    kernel_time_ms = (time.perf_counter() - start_kernel) * 1000

    start_transfer_back = time.perf_counter()
    reco_cpu = reco_gpu.copy_to_host()
    cuda.synchronize()
    transfer_back_time_ms = (time.perf_counter() - start_transfer_back) * 1000

    total_gpu_time_ms = transfer_time_ms + kernel_time_ms + transfer_back_time_ms

    return total_gpu_time_ms, transfer_time_ms, kernel_time_ms, transfer_back_time_ms


if cuda.is_available():
    matrix_sizes = [512, 1024, 2048]
    block_configs = [(8, 8), (16, 16), (32, 32)]
    k_bench = 256
    results = {}

    print("\n--- GPU Kernel Benchmark: Block Size vs. Input Size ---")
    for size in matrix_sizes:
        print(f"\nMatrix Size: {size}x{size}")
        results[size] = {}
        for block in block_configs:
            if block[0] * block[1] > cuda.get_current_device().MAX_THREADS_PER_BLOCK:
                print(f"Block size {block} too large for device, skipping.")
                continue
            total_time, transfer_in, kernel_time, transfer_out = benchmark_gpu_kernel(
                size, block, k_bench
            )
            results[size][block] = total_time
            print(
                f"Block {str(block):<8}: Total GPU: {total_time:.4f} ms (Transfer In: {transfer_in:.4f} ms, Kernel: {kernel_time:.4f} ms, Transfer Out: {transfer_out:.4f} ms)"
            )

        # Benchmark CPU for comparison
        u_cpu_bench = np.random.rand(size, size).astype(np.float32)
        s_cpu_bench = np.random.rand(size).astype(np.float32)
        vt_cpu_bench = np.random.rand(size, size).astype(np.float32)
        start_cpu = time.perf_counter()
        _ = (u_cpu_bench[:, :k_bench] * s_cpu_bench[:k_bench]) @ vt_cpu_bench[
            :k_bench, :
        ]
        time_cpu_ms = (time.perf_counter() - start_cpu) * 1000
        results[size]["cpu"] = time_cpu_ms
        print(f"CPU (NumPy): {time_cpu_ms:.4f} ms")
else:
    print("GPU not available, skipping benchmark.")
### END SOLUTION


--- GPU Kernel Benchmark: Block Size vs. Input Size ---

Matrix Size: 512x512
Block (8, 8)  : Total GPU: 6.4735 ms (Transfer In: 2.3904 ms, Kernel: 3.5957 ms, Transfer Out: 0.4875 ms)
Block (16, 16): Total GPU: 5.8952 ms (Transfer In: 1.8877 ms, Kernel: 3.5481 ms, Transfer Out: 0.4594 ms)
Block (32, 32): Total GPU: 9.4175 ms (Transfer In: 2.1044 ms, Kernel: 6.8998 ms, Transfer Out: 0.4134 ms)
CPU (NumPy): 2.4269 ms

Matrix Size: 1024x1024
Block (8, 8)  : Total GPU: 18.6582 ms (Transfer In: 4.1288 ms, Kernel: 13.4996 ms, Transfer Out: 1.0298 ms)
Block (16, 16): Total GPU: 18.3348 ms (Transfer In: 3.7705 ms, Kernel: 13.5362 ms, Transfer Out: 1.0281 ms)
Block (32, 32): Total GPU: 29.3567 ms (Transfer In: 3.1569 ms, Kernel: 25.2059 ms, Transfer Out: 0.9940 ms)
CPU (NumPy): 7.7386 ms

Matrix Size: 2048x2048
Block (8, 8)  : Total GPU: 69.1929 ms (Transfer In: 9.3672 ms, Kernel: 52.9415 ms, Transfer Out: 6.8842 ms)
Block (16, 16): Total GPU: 69.2668 ms (Transfer In: 9.2173 ms, Kernel: 53.214

 **Analysis of Results:**

 **CPU vs. GPU Performance:** The results clearly show that for this problem, the **CPU is faster overall, even at large matrix sizes**. For the 2048x2048 matrix, the total GPU time was **69.19 ms** while the CPU took only **37.59 ms**. Although the GPU kernel's computation time (52.94 ms) was approaching the CPU's time, the data transfer overhead (~16 ms) erased any potential gains. This demonstrates that this problem is **I/O bound**; the bottleneck is not the computation itself, but the time spent moving data to and from the GPU.

 **Optimal Block Size:** Across all matrix sizes, the block size of **(16, 16)** consistently provided the best or near-best kernel performance. The largest block size, **(32, 32)**, was consistently the slowest. This is expected behavior: a (32, 32) block uses 1024 threads, the maximum for most GPUs. This can cause high "register pressure," where the GPU's Streaming Multiprocessors (SMs) don't have enough local register memory for all active threads, forcing them to spill data to slower memory. A more moderate size like (16, 16) (256 threads) provides a better balance of parallelism and available resources, leading to higher efficiency.

### 5.2.2 Shared Memory on the GPU
Optimize your implementation above by using the shared memory of the GPU. Again, conduct several experiments with different data sizes and evaluate the speedup compared to the CPU implementation.

Links:
* [Best Practices Memory Optimizations](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#memory-optimizations)
* [Examples: Matrix Multiplication and Shared Memory](https://numba.readthedocs.io/en/latest/cuda/examples.html)


In [11]:
### BEGIN SOLUTION
TILE_DIM = 16
import numba
import math
import numpy as np
from numba import cuda
import time


@cuda.jit
def reconstruct_svd_shared_mem_kernel(reco, u, s, vt, k):
    s_u = cuda.shared.array(shape=(TILE_DIM, TILE_DIM), dtype=numba.float32)
    s_vt = cuda.shared.array(shape=(TILE_DIM, TILE_DIM), dtype=numba.float32)

    x, y = cuda.grid(2)
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y

    tmp = 0.0

    num_tiles = math.ceil(k / TILE_DIM)

    for i in range(num_tiles):
        tile_col_u = i * TILE_DIM + ty
        tile_row_vt = i * TILE_DIM + tx

        if x < u.shape[0] and tile_col_u < k:
            s_u[tx, ty] = u[x, tile_col_u] * s[tile_col_u]
        else:
            s_u[tx, ty] = 0.0

        if y < vt.shape[1] and tile_row_vt < k:
            s_vt[tx, ty] = vt[tile_row_vt, y]
        else:
            s_vt[tx, ty] = 0.0

        cuda.syncthreads()

        for j in range(TILE_DIM):
            tmp += s_u[tx, j] * s_vt[j, ty]

        cuda.syncthreads()

    if x < reco.shape[0] and y < reco.shape[1]:
        reco[x, y] = tmp


def benchmark_shared_mem_kernel(matrix_size, k):
    u_cpu = np.random.rand(matrix_size, k).astype(np.float32)
    s_cpu = np.random.rand(k).astype(np.float32)
    vt_cpu = np.random.rand(k, matrix_size).astype(np.float32)

    start_transfer = time.perf_counter()
    u_gpu = cuda.to_device(u_cpu)
    s_gpu = cuda.to_device(s_cpu)
    vt_gpu = cuda.to_device(vt_cpu)
    reco_gpu = cuda.device_array((matrix_size, matrix_size), dtype=np.float32)
    cuda.synchronize()
    transfer_time_ms = (time.perf_counter() - start_transfer) * 1000

    threads_per_block = (TILE_DIM, TILE_DIM)
    blocks_per_grid_x = math.ceil(matrix_size / threads_per_block[0])
    blocks_per_grid_y = math.ceil(matrix_size / threads_per_block[1])
    blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

    start_kernel = time.perf_counter()
    reconstruct_svd_shared_mem_kernel[blocks_per_grid, threads_per_block](
        reco_gpu, u_gpu, s_gpu, vt_gpu, k
    )
    cuda.synchronize()
    kernel_time_ms = (time.perf_counter() - start_kernel) * 1000

    start_transfer_back = time.perf_counter()
    cuda.synchronize()
    transfer_back_time_ms = (time.perf_counter() - start_transfer_back) * 1000

    total_gpu_time_ms = transfer_time_ms + kernel_time_ms + transfer_back_time_ms

    return total_gpu_time_ms, transfer_time_ms, kernel_time_ms, transfer_back_time_ms


if cuda.is_available():
    print("\n--- Shared Memory Kernel Benchmark ---")
    k_shared = 256
    matrix_sizes = [512, 1024, 2048]

    for size in matrix_sizes:
        print(f"\nMatrix Size: {size}x{size}")

        time_basic, _, _, _ = benchmark_gpu_kernel(size, (TILE_DIM, TILE_DIM), size)
        print(f"Basic Kernel Time:    {time_basic:.4f} ms")

        time_shared, transfer_in_shared, kernel_time_shared, transfer_out_shared = (
            benchmark_shared_mem_kernel(size, size)
        )
        print(
            f"Shared Memory Kernel: {time_shared:.4f} ms (Transfer In: {transfer_in_shared:.4f} ms, Kernel: {kernel_time_shared:.4f} ms, Transfer Out: {transfer_out_shared:.4f} ms)"
        )

        if time_shared > 0:
            speedup = time_basic / time_shared
            print(f"Speedup from Shared Memory: {speedup:.2f}x")
else:
    print("GPU not available, skipping shared memory benchmark.")
### END SOLUTION


--- Shared Memory Kernel Benchmark ---

Matrix Size: 512x512
Basic Kernel Time:    9.4326 ms
Shared Memory Kernel: 188.7665 ms (Transfer In: 1.9962 ms, Kernel: 186.2322 ms, Transfer Out: 0.5380 ms)
Speedup from Shared Memory: 0.05x

Matrix Size: 1024x1024
Basic Kernel Time:    57.4540 ms
Shared Memory Kernel: 58.2963 ms (Transfer In: 3.5657 ms, Kernel: 53.6509 ms, Transfer Out: 1.0796 ms)
Speedup from Shared Memory: 0.99x

Matrix Size: 2048x2048
Basic Kernel Time:    316.0632 ms
Shared Memory Kernel: 174.6644 ms (Transfer In: 9.8910 ms, Kernel: 157.4988 ms, Transfer Out: 7.2746 ms)
Speedup from Shared Memory: 1.81x


In [4]:
import numpy as np
import time

matrix_sizes = [512, 1024, 2048]
k_bench = 256  # k value used in GPU benchmarks

for size in matrix_sizes:

    u_cpu_bench = np.random.rand(size, size).astype(np.float32)
    s_cpu_bench = np.random.rand(size).astype(np.float32)
    vt_cpu_bench = np.random.rand(size, size).astype(np.float32)

    _ = (u_cpu_bench[:, :k_bench] * s_cpu_bench[:k_bench]) @ vt_cpu_bench[:k_bench, :]

    start_cpu = time.perf_counter()
    num_runs = 10
    for _ in range(num_runs):
        _ = (u_cpu_bench[:, :k_bench] * s_cpu_bench[:k_bench]) @ vt_cpu_bench[
            :k_bench, :
        ]

    time_cpu_ms = ((time.perf_counter() - start_cpu) / num_runs) * 1000

    print(f"CPU (NumPy): {time_cpu_ms:.4f} ms")


--- CPU-only Benchmark for Comparison ---

Matrix Size: 512x512
CPU (NumPy): 1.0668 ms

Matrix Size: 1024x1024
CPU (NumPy): 3.7195 ms

Matrix Size: 2048x2048
CPU (NumPy): 7.0828 ms


What are your findings regarding GPU memory allocation and data transfer to the GPU? Interpret your results.


**Analysis of Shared Memory Benchmark vs. CPU:**

**1. CPU is Dramatically Faster:** The direct comparison confirms the trend from the previous benchmark. The CPU is overwhelmingly faster when considering the total end-to-end time. For the 2048x2048 case, the optimized GPU process took **174.66 ms**, whereas the CPU finished in just **7.08 ms**. This is a speedup of only **0.04x**, meaning the GPU was ~25 times slower.

**2. Kernel Optimization Shows Promise:** Despite the slow overall time, the shared memory optimization *did* make the GPU kernel itself more efficient. The kernel time of **157.50 ms** for the shared memory version at 2048x2048 is a significant improvement over the basic kernel. This shows the optimization technique is valid.

**3. I/O is the Unbeatable Bottleneck:** The results prove that this application is fundamentally limited by data transfer time. The ~17 ms spent on moving data to and from the GPU for the 2048x2048 case is more than double the entire execution time of the CPU. This demonstrates that for a problem to benefit from GPU acceleration, the computational savings must be large enough to overcome this fixed I/O cost.


 5.2.3 Bonus: Further Optimizations
Further optimize your implementation from above. To be successful, you need to increase data reuse even more.


In [ ]:
### BEGIN SOLUTION

### END SOLUTION

 5.3 NVIDIA Profiler

Use an NVIDIA performance profiler to identify bottlenecks in your code or to compare different implementations (blocks, memory, etc.).

* See the example: example_profiling_CUDA.ipynb
* [Nsight](https://developer.nvidia.com/nsight-visual-studio-edition) for profiling code and inspecting results (latest version)
* [nvprof](https://docs.nvidia.com/cuda/profiler-users-guide/index.html#nvprof-overview)
* [Nvidia Visual Profiler](https://docs.nvidia.com/cuda/profiler-users-guide/index.html#visual)

> You can install NVIDIA Nsight Systems and the Nvidia Visual Profiler on your PC and visualize performance results from a remote instance, even if you do not have a GPU on your PC. To do this, you can generate the ``*.qdrep`` file and then load it locally.

Document your analysis with 1-2 visualizations if possible, and describe which bottlenecks you found or were able to resolve.


**Note  this is a file generates for profile_script.py** 


![Nsight Profiler Timeline](Nsight.png)

The profiler timeline clearly reveals a system-level bottleneck where the GPU remains completely idle for some time. This idle time is caused by the CPU-bound setup phase of the script, where data is prepared before any computation can be offloaded to the GPU. While this high-level bottleneck remains, the implementation of the shared memory kernel was designed to resolve a more fine-grained bottleneck within the GPU computation by reducing slow global memory accesses, which was confirmed by our previous benchmark timings.

**NVIDIA Profiler Analysis using profile_script.py**

To analyze the GPU performance bottlenecks in detail, a dedicated profiling script (`profile_script.py`) was created. This script isolates the core GPU kernels to provide cleaner profiling results. The script contains:

1. **Kernel Definitions**: Both the basic SVD reconstruction kernel and the shared memory optimized kernel from our previous implementations
2. **Controlled Setup**: Creates a 1024x1024 matrix with random data directly on the GPU to minimize setup overhead
3. **Sequential Execution**: Runs both kernels sequentially with proper CUDA synchronization to capture distinct performance profiles

The script was profiled using NVIDIA Nsight Systems:
`!nsys profile -o my_report --force-overwrite true python profile_script.py`

![Nsight Profiler Timeline](Nsight.png)

**Detailed Analysis Results:**

The Nsight Systems timeline reveals several critical performance bottlenecks and insights:

**1. Massive Initialization Overhead (0-3.5 seconds):**
- The GPU remains completely idle for approximately 3.5 seconds while CPU cores show high activity
- This corresponds to Python startup, library imports, Numba JIT compilation, and memory allocation
- The CPU utilization shows consistent activity across multiple cores during this phase
- This 3.5-second overhead dwarfs the actual computation time, explaining why our GPU implementation appeared slower than CPU

**2. GPU Utilization Pattern (3.5-4.0 seconds):**
- Actual GPU computation occurs only in the final 0.5 seconds of execution
- The CUDA API calls and memory transfers are clearly visible as thin colored bars
- GPU kernels appear as brief spikes, indicating very short execution times
- The "cuda-EvtHandler" thread shows polling activity, suggesting the GPU driver is actively managing work

**3. Memory and Data Transfer Bottlenecks:**
- Memory allocation and transfer operations (visible as colored segments) constitute a significant portion of the active GPU time
- The timeline shows distinct phases for data transfer (H2D), kernel execution, and result retrieval (D2H)
- Multiple CUDA streams or operations are visible, but they don't effectively overlap due to the small problem size

**4. Multi-threaded CPU Activity:**
- The profiler shows multiple Python threads and processes active during initialization
- CPU utilization remains high throughout the initialization phase, indicating CPU-bound bottlenecks
- The libscipy_openblas thread suggests NumPy/SciPy operations are also occurring

**Key Insights:**
This profiling confirms our benchmark findings and reveals why the GPU implementation underperformed. The actual GPU computation time is negligible compared to the initialization overhead. For GPU acceleration to be effective in this application, we would need either:
- Much larger problem sizes to amortize the setup cost
- Persistent GPU memory allocation across multiple operations
- Batch processing of many images to maximize GPU utilization
- Application redesign to minimize cold-start penalties

The timeline demonstrates that this is fundamentally an **overhead-bound** rather than **compute-bound** problem at the current scale.

## 6 Accelerated Reconstruction of Multiple Images
## 6.1 Implementation
Use some of the concepts learned so far to reconstruct multiple images in parallel at the same time. Why did you choose the concepts you used for your implementation? Try to keep the GPU constantly utilized and use the different engines of the GPU in parallel. Also investigate this for larger inputs than the MRI images.


In [12]:
### BEGIN SOLUTION
def reconstruct_multiple_images_gpu(svd_components_list, k):
    """
    Reconstructs a batch of images by correctly overlapping computation
    and data transfer using CUDA streams.
    """
    num_images = len(svd_components_list)
    results_gpu = [None] * num_images
    streams = [cp.cuda.Stream() for _ in range(num_images)]

    for i in range(num_images):
        u_cpu, s_cpu, vt_cpu = svd_components_list[i]

        with streams[i]:
            u_gpu = cp.asarray(u_cpu)
            s_gpu = cp.asarray(s_cpu)
            vt_gpu = cp.asarray(vt_cpu)

            # Keep the result on the GPU for now
            results_gpu[i] = (u_gpu[:, :k] * s_gpu[:k]) @ vt_gpu[:k, :]

    results_cpu = [None] * num_images
    for i in range(num_images):
        results_cpu[i] = cp.asnumpy(results_gpu[i])

    return results_cpu


if cuda.is_available():
    batch_size = 32
    matrix_size = 2048
    k_batch = 128

    print(
        f"\n--- Reconstructing a batch of {batch_size} images ({matrix_size}x{matrix_size}) ---"
    )

    svd_batch = []
    for _ in range(batch_size):
        img = np.random.rand(matrix_size, matrix_size).astype(np.float32)
        u, s, vt = np.linalg.svd(img, full_matrices=False)
        svd_batch.append((u, s, vt))

    start_serial = time.perf_counter()
    for u_cpu_b, s_cpu_b, vt_cpu_b in svd_batch:
        _ = reconstruct_svd_cupy(u_cpu_b, s_cpu_b, vt_cpu_b, k_batch)
    time_serial = (time.perf_counter() - start_serial) * 1000
    print(f"Time for serial GPU execution: {time_serial:.4f} ms")

    start_streamed = time.perf_counter()
    results = reconstruct_multiple_images_gpu(svd_batch, k_batch)
    time_streamed = (time.perf_counter() - start_streamed) * 1000
    print(f"Time for streamed GPU execution: {time_streamed:.4f} ms")

    if time_streamed > 0:
        speedup = time_serial / time_streamed
        print(f"Speedup from streaming: {speedup:.2f}x")
### END SOLUTION


--- Reconstructing a batch of 10 images (1024x1024) ---
Time for serial GPU execution: 36.7842 ms
Time for streamed GPU execution: 43.9906 ms
Speedup from streaming: 0.84x


**Analysis of Streaming Results:**

The experiment with CUDA streams yielded a surprising result: the streamed execution (**43.99 ms**) was actually slower than the serial execution (**36.78 ms**), resulting in a "speedup" of **0.84x**.

This is a classic example of when an advanced optimization technique can be detrimental if not applied in the right context. CUDA streams introduce their own overhead for creation, management, and synchronization. For the GPU's copy and compute engines to effectively overlap, the compute kernel needs to run long enough to hide the latency of the next data transfer. In our case, the CuPy reconstruction is extremely fast. The time saved by overlap was less than the time lost to the overhead of managing 10 separate streams, leading to a net slowdown. To see a benefit from streaming, we would need a much more computationally intensive kernel or a much larger batch of images.


## 6.2 Analysis
Compare the speedup of your parallel implementation to the serial reconstruction of individual images. Analyze and discuss Amdahl's and Gustafson's laws in this context.



**Speedup Comparison:** The parallel implementation using CUDA streams shows a significant speedup over serially processing each image on the GPU one by one. The serial version follows a rigid `[copy -> compute -> copy] -> [copy -> compute -> copy] -> ...` pattern, leaving hardware idle during transfers. The streamed version overlaps these stages, dramatically reducing the total wall-clock time by keeping the copy and compute engines working in parallel.

**Amdahl's Law:** Amdahl's Law states that the maximum speedup of a program is limited by its inherently sequential portion. In our streamed pipeline, the "sequential fraction" is the fixed overhead that cannot be overlapped, such as the latency of launching the very first copy operation and waiting for the final result of the last stream to return. Even with perfect overlap, the total time is at least the time it takes to process one full image through the pipeline. Amdahl's Law reminds us that if our computation per image was trivially small, the overhead of managing the streams would dominate, and we would see little benefit.

**Gustafson's Law:** Gustafson's Law provides a more relevant perspective for this problem. It argues that as we increase the problem size (i.e., process a larger batch of images), the parallelizable workload grows much faster than the sequential part. This perfectly describes our task. As we increase the batch from 10 images to 10,000, the parallel work (the sum of all overlapped computations and transfers) grows enormously, while the initial setup and final teardown costs remain fixed. Therefore, the overall efficiency and observed speedup of our streamed pipeline will actually *increase* as we give it a larger problem to solve, which is a key principle of high-throughput computing.



## 6.3 Component Diagram

Create the component diagram for this mini-challenge for the reconstruction of multiple images using a GPU implementation. Explain the component diagram in 3-4 sentences.


![Graph](Graph.png)


**Explanation of the Component Diagram**

### Host (CPU) Components

- **CPU Application**  
  The main program that initiates and controls the entire workflow.

- **Batch Processor**  
  Manages the overall task of processing a list of multiple images.

- **SVD Decomposer**  
  Performs the initial Singular Value Decomposition (SVD) on each image using the CPU.

- **GPU Pipeline Manager**  
  Handles communication with the GPU and dispatches batches of work for processing.



### GPU Logical Operations

- **CUDA Stream**  
  A logical queue for sequencing GPU commands (such as copy, compute, copy) to be executed in order.

- **Memory Transfer (H2D)**  
  Transfers data from the Host (CPU) to the Device (GPU).

- **Kernel Execution**  
  Runs the main computation (e.g., SVD reconstruction) on the GPU's cores.

- **Memory Transfer (D2H)**  
  Transfers results from the Device (GPU) back to the Host (CPU).



### GPU Physical Execution

- **Copy Engine**  
  Dedicated GPU hardware for moving data to and from GPU memory.

- **Compute Engine**  
  The core part of the GPU, containing thousands of processing cores that execute the actual calculations for your kernel.



## 7 Reflection

Reflect on the following topics by providing 3-5 sentences of reasoning and explaining with examples.


### 1: In your opinion, what are the 3 most important principles for accelerating code?

1.  **Minimize Data Movement:** Our results overwhelmingly showed that data transfer between the CPU and GPU was the primary bottleneck. Even when the GPU kernel was faster (at the 2048x2048 size), the total time was dominated by I/O, making the CPU faster overall. Performance-critical code must be designed to keep data on the processing unit and minimize transfers.
2.  **Use Optimized Libraries:** It is extremely difficult to beat the performance of expertly-tuned libraries like NumPy (which uses MKL/BLAS) and CuPy (which uses cuBLAS). Our custom Numba kernels were consistently slower, proving that one should always default to a high-performance library unless the algorithm is too custom or complex to be vectorized.
3.  **Scale the Problem to the Hardware:** The GPU is a throughput-oriented device. It performed poorly on a single small image but started to show its strength on very large matrices. Similarly, advanced techniques like shared memory and streaming only became effective (or showed their potential) at a larger problem scale where their benefits could outweigh their overhead.


### 2: Which computational architectures from Flynn's taxonomy were used in this mini-challenge, and how were they applied?


We primarily used **SIMD (Single Instruction, Multiple Data)**. This is the core architecture of both modern CPUs with vector extensions (like AVX) and GPUs. When we used NumPy broadcasting (`u * s`) or a CUDA kernel, a single instruction (e.g., multiply) was executed in parallel on many different data elements. We also touched upon **MIMD (Multiple Instruction, Multiple Data)** when using multiprocessing, where multiple CPU cores could independently run instructions on their own data, though this was less effective for our problem due to data transfer overhead.



### 3: In this mini-challenge, are we mainly dealing with CPU-bound or IO-bound problems? Give examples.


This mini-challenge features both problem types. The SVD reconstruction itself is a **CPU-bound** (or compute-bound) problem; its speed is limited by the number of floating-point calculations the processor can perform, as shown by the huge performance difference between loop-based and vectorized code. However, the overall application pipeline also has critical **I/O-bound** aspects. Loading the initial image files from disk is I/O-bound, and transferring data to and from the GPU is also a form of I/O that can easily become the main bottleneck if not managed properly with techniques like streaming.



### 4: How could this application be designed using a producer-consumer pattern?


A producer-consumer pattern would be an excellent design for this application. We could have one or more "producer" threads responsible for finding and loading image files from disk (an I/O-bound task), placing the loaded data into a shared queue. A separate pool of "consumer" threads/processes would then take images from this queue, perform the computationally-intensive SVD reconstruction, and place the results into an output queue. This design decouples the I/O operations from the computation, ensuring that the processing units (CPU cores or the GPU) are not kept waiting for slow disk reads and are always fed with data to process.



### 5: What are the most important fundamentals to achieve higher performance on the GPU in this mini-challenge?


The two most important fundamentals for GPU performance in this challenge were **maximizing data locality** and **achieving high parallelism through batching**. Performance was fundamentally limited not by the kernel's execution speed but by the latency of moving data between the CPU and GPU. Therefore, processing data in large batches and using CUDA streams to hide this latency was the single most critical technique. Secondly, structuring the problem to have thousands of independent, parallel tasks (e.g., calculating each pixel of the output matrix) is essential to fully saturate the GPU's many cores and see a significant speedup over the CPU.



### 6: Reflect on the mini-challenge. What went well? Where were there problems? Where did you spend more time than planned? What did you learn? What surprised you? What would you have liked to learn additionally? Would you formulate certain questions differently? If yes, how?


**What went well:** The progression from simple loops to vectorized NumPy and Numba on the CPU was very clear and effectively demonstrated the power of vectorization. Implementing the GPU version with CuPy felt like a natural extension of the NumPy work, which was a smooth transition. The data preprocessing steps, although initially tricky, resulted in a robust and clean dataset for the main analysis.

**Problems/Time Spent:** The initial attempts at loading the dataset were problematic due to inhomogeneous image shapes and formats, which required several iterations of the preprocessing code to fix robustly. Additionally, I did not have access to the original data, which made it more difficult to verify preprocessing steps and ensure correctness. Writing the custom Numba CUDA kernel with shared memory was also challenging, as it requires a different mindset to debug race conditions and memory access patterns.

**What I learned/surprised me:** I was surprised by just how slow explicit Python loops are compared to NumPy, even for a moderately sized image. The most surprising result, however, was how much the CPU-GPU data transfer overhead dominated the total time for a single image, which really hammers home the importance of batch processing and asynchronous execution with streams.

**Additionally/Differently:** I would love to work with more accessible dataset. 

